In [ ]:
//%use intellij-sdk
USE {
    repositories("https://packages.jetbrains.team/maven/p/kds/kotlin-ds-maven")
    dependencies("org.jetbrains.kotlinx:kotlin-jupyter-intellij-platform:0.0.2-7.dev1")
}


# Notebooks & IntelliJ Platform SDK Demo


<img alt="nb-logo.svg" src="nb-logo.svg" width="150"/>  <img alt="idea.png" src="idea.png" width="150"/>


🎯 Welcome to the IntelliJ Platform SDK demonstration! This notebook shows how to interact with IntelliJ IDEA's core functionality using Kotlin, areas covered:

* file system manipulation
* UI elements creation
* document editing

In [27]:
val projectRoot = "/Users/Nikolay.Egorov/Desktop/Work/KotlinConf2025Demo"
val sampleName = "$projectRoot/data/ex.kts"

In [32]:
val txt = getDocumentByPath("$projectRoot/data/ex.kts")

txt

DocumentImpl[ex.kts]

In [ ]:
import com.intellij.openapi.editor.Document
import com.intellij.openapi.fileEditor.FileDocumentManager
import com.intellij.openapi.vfs.LocalFileSystem
import com.intellij.openapi.vfs.VfsUtilCore
import com.intellij.openapi.vfs.VirtualFile

// Let's create a helper function to work with virtual files
fun createOrGetVirtualFile(path: kotlin.String) : VirtualFile? =
    LocalFileSystem.getInstance().refreshAndFindFileByPath(path)

fun getDocumentByPath(path: kotlin.String) : Document? {
    val file = createOrGetVirtualFile(path) ?: return null
    return FileDocumentManager.getInstance().getDocument(file)
}

fun VirtualFile.readText() = VfsUtilCore.loadText(this)

In [ ]:
import com.intellij.openapi.command.WriteCommandAction
import com.intellij.openapi.editor.Document
import com.intellij.openapi.fileEditor.FileDocumentManager

fun Document.setContent(textToAdd: kotlin.String, rewrite: Boolean = false): Boolean {
    val existingText = text
    val updatedText = if (!rewrite && existingText.isNotEmpty()) {
        "$existingText\n$textToAdd"
    } else {
        textToAdd
    }

    WriteCommandAction.runWriteCommandAction(null) {
        this.setText(updatedText)
        FileDocumentManager.getInstance().saveDocument(this)
    }

    return true
}

In [ ]:
txt?.appendToDocument("""
    println("Hello KotlinConf 2025!")

    // TODO: Add more exciting code here
""".trimIndent())



Let's create another cell to test this function:



In [ ]:
// Let's demonstrate how to analyze file structure
fun analyzeFile(filePath: String) {
    val vFile = createOrGetVirtualFile(filePath)
    val doc = FileDocumentManager.getInstance().getDocument(vFile)

    println("File analysis:")
    println("- Name: ${vFile.name}")
    println("- Extension: ${vFile.extension}")
    println("- Size: ${vFile.length} bytes")
    println("- Lines: ${doc?.lineCount ?: 0}")
}

analyzeFile("data/demo.kt")

# Dialog window creation 



In [23]:
import com.intellij.openapi.ui.DialogWrapper
import com.intellij.openapi.ui.ValidationInfo
import com.intellij.ui.components.JBTextField
import com.intellij.ui.dsl.builder.panel
import com.intellij.openapi.fileChooser.FileChooserDescriptor
import com.intellij.openapi.fileChooser.FileChooserDialog
import com.intellij.openapi.fileChooser.FileChooserFactory
import com.intellij.openapi.ui.setEmptyState
import com.intellij.ui.util.preferredHeight
import javax.swing.JComponent
import java.io.File

class CreateFileDialogWithChooser : DialogWrapper(true) {
    private val fileNameField = JBTextField()
    private val locationField = JBTextField()

    init {
        title = "Create New File"
        fileNameField.setEmptyState("Enter file name")
        fileNameField.emptyText
        fileNameField.setSize(fileNameField.emptyText.preferredSize)
        locationField.text = projectRoot // Using existing projectRoot variable
        init()
    }

    private fun showFileChooser() {
        val descriptor = FileChooserDescriptor(false, true, false, false, false, false)
            .withTitle("Select Directory")
            .withDescription("Choose location for new file")

        val chooser = FileChooserFactory.getInstance()
            .createFileChooser(descriptor, null, null)

        val files = chooser.choose(null)
        if (files.isNotEmpty()) {
            locationField.text = files[0].path
        }
    }

    override fun createCenterPanel(): JComponent = panel {
        row("File Name:") { cell(fileNameField).resizableColumn() }
        row("Location:") {
            cell(locationField)
            button("Browse") { showFileChooser() }
        }
    }

    override fun doValidate(): ValidationInfo? {
        if (fileNameField.text.isBlank()) {
            return ValidationInfo("File name cannot be empty", fileNameField)
        }
        if (locationField.text.isBlank()) {
            return ValidationInfo("Location cannot be empty", locationField)
        }
        return null
    }

    fun getFileName() = fileNameField.text
    fun getPath() = locationField.text
}


In [24]:
import com.intellij.openapi.application.invokeLater

invokeLater { CreateFileDialogWithChooser().showAndGet() }

In [15]:
import com.intellij.openapi.command.WriteCommandAction

fun showCreateFileDialogWithChooser() {
    val dialog = CreateFileDialogWithChooser()
    if (dialog.showAndGet()) {
        val fileName = dialog.getFileName()
        val location = dialog.getPath()
        val fullPath = "$location/${fileName}"

        val file = createOrGetVirtualFile(fullPath)
        if (file != null) {
            println("File already exists")
            return
        }

        WriteCommandAction.runWriteCommandAction(null) {
            val newFile = LocalFileSystem.getInstance().refreshAndFindFileByPath(location)
                ?.createChildData(null, fileName)
            if (newFile != null) {
                println("File created successfully at: ${newFile.path}")
            } else {
                println("Failed to create file")
            }
        }
    }
}

// Show the dialog with file chooser
invokeLater {
    showCreateFileDialogWithChooser()
}

This  version includes:
1. FileChooserDescriptor for directory selection
2. "Browse" button that opens native file chooser
3. Uses VirtualFile API for file creation
4. Pre-fills location with projectRoot
5. Proper file existence check before creation

# Adding Syntax Highlighting to Editor
To enable syntax highlighting in the editor, we need to modify the TextEditorDialog to use EditorEx and configure the editor with proper file type and highlighting settings.

In [33]:
import com.intellij.openapi.ui.DialogWrapper
import com.intellij.ui.components.JBScrollPane
import com.intellij.ui.EditorTextField
import com.intellij.openapi.editor.ex.EditorEx
import com.intellij.openapi.fileTypes.FileTypeManager
import java.awt.Dimension
import javax.swing.JComponent
import com.intellij.openapi.command.WriteCommandAction
import com.intellij.openapi.fileEditor.FileDocumentManager

class TextEditorDialog(private val file: VirtualFile) : DialogWrapper(true) {
    private val textArea = EditorTextField(file.readText(), null, file.fileType)

    init {
        title = "Edit File Content"
        init()
    }

    override fun createCenterPanel(): JComponent {
        textArea.setOneLineMode(false)
        textArea.setPreferredSize(Dimension(500, 300))

        // Configure editor to enable syntax highlighting
        (textArea.editor as? EditorEx)?.let { editor ->
            editor.setHorizontalScrollbarVisible(true)
            editor.setVerticalScrollbarVisible(true)
            editor.settings.apply {
                isLineNumbersShown = true
                isAutoCodeFoldingEnabled = true
            }
        }

        return JBScrollPane(textArea)
    }

    fun getContent() = textArea.text
}

invokeLater {
    val f = createOrGetVirtualFile(sampleName)!!

    val dialog = TextEditorDialog(createOrGetVirtualFile(sampleName)!!)
    if (dialog.showAndGet()) {
        WriteCommandAction.runWriteCommandAction(null) {
            val document = FileDocumentManager.getInstance().getDocument(f)
            document?.setText(dialog.getContent())
            FileDocumentManager.getInstance().saveDocument(document!!)
        }
    }
}


This enhanced version:
1. Creates editor with proper file type detection for syntax highlighting
2. Enables line numbers
3. Adds proper scrollbars
4. Configures code folding


In [31]:
invokeLater {
    TextEditorDialog(createOrGetVirtualFile(sampleName)!!).showAndGet()
}